# This is a Jupyter Notebook

In [ ]:
"""
visualkeras: transformers example
==========================================

An example showing the :py:func:`~scikitplot.visualkeras` function
used by a :py:class:`~tensorflow.keras.Model` or :py:class:`~torch.nn.Module` or
:py:class:`~transformers.TFPreTrainedModel` model.

.. important::

    ⚠️ Deprecated Transformers models are not supported in TensorFlow — use KerasNLP or KerasHub instead.
    `🚫 transformers deprecated models <https://www.linkedin.com/feed/update/urn:li:activity:7338966863403528192/>`_.
"""

'\nvisualkeras: transformers example\n==========================================\n\nAn example showing the :py:func:`~scikitplot.visualkeras` function\nused by a :py:class:`~tensorflow.keras.Model` or :py:class:`~torch.nn.Module` or\n:py:class:`~transformers.TFPreTrainedModel` model.\n\n.. important::\n\n    ⚠️Deprecated transformers models are not supported to tensorflow use keras-nlp or keras-hub.\n    `🚫 transformers deprecated models <https://www.linkedin.com/feed/update/urn:li:activity:7338966863403528192/>`_.\n'

In [2]:

# pip install protobuf==5.29.4
import tensorflow as tf

# Clear any session to reset the state of TensorFlow/Keras
tf.keras.backend.clear_session()

from transformers import TFAutoModel, AutoModel

from scikitplot import visualkeras

2025-08-14 21:20:39.319164: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-14 21:20:39.339226: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-14 21:20:39.382313: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755206439.428047   92929 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755206439.440232   92929 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755206439.505016   92929 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

0it [00:00, ?it/s]

In [3]:

# Load the Hugging Face transformer model
# transformer_model = AutoModel.from_pretrained("microsoft/mpnet-base")
# transformer_model = TFAutoModel.from_pretrained("microsoft/mpnet-base")
transformer_model = TFAutoModel.from_pretrained("bert-base-uncased")

/root/micromamba/envs/py311/lib/python3.11/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


RuntimeError: Failed to import transformers.models.bert.modeling_tf_bert because of the following error (look up to see its traceback):
No module named 'keras.__internal__'

In [4]:

# Define a Keras-compatible wrapper for the Hugging Face model
def wrap_transformer_model(inputs):
    input_ids, attention_mask = inputs
    outputs = transformer_model(input_ids=input_ids, attention_mask=attention_mask)
    return outputs.last_hidden_state  # Return the last hidden state for visualization

In [5]:

# Define Keras model inputs
input_ids = tf.keras.Input(shape=(128,), dtype=tf.int32, name="input_ids")
attention_mask = tf.keras.Input(shape=(128,), dtype=tf.int32, name="attention_mask")

# Pass inputs through the transformer model using a Lambda layer
seq_len = transformer_model.config.max_position_embeddings  # usually 512 for BERT, here you used 128
hidden_size = transformer_model.config.hidden_size
last_hidden_state = tf.keras.layers.Lambda(
    wrap_transformer_model,
    output_shape=(seq_len, hidden_size),  # Explicitly specify the output shape
    name="bert-base-uncased",
)([input_ids, attention_mask])

# Reshape the output to fit into Conv2D (adding extra channel dimension) inside a Lambda layer
# def reshape_last_hidden_state(x):
#     return tf.reshape(x, (-1, 1, 128, 768))
# reshaped_output = tf.keras.layers.Lambda(reshape_last_hidden_state)(last_hidden_state)
# Use Reshape layer to reshape the output to fit into Conv2D (adding extra channel dimension)
# Reshape to (batch_size, 128, 768, 1) for Conv2D input
# reshaped_output = tf.keras.layers.Reshape((seq_len, hidden_size, 1))(last_hidden_state)
reshaped_output = tf.keras.layers.Reshape((-1, seq_len, hidden_size))(last_hidden_state)

# Add different layers to the model
x = tf.keras.layers.Conv2D(
    512, (3, 3), activation="relu", padding="same", name="conv2d_1"
)(reshaped_output)
x = tf.keras.layers.BatchNormalization(name="batchnorm_1")(x)
x = tf.keras.layers.Dropout(0.3, name="dropout_1")(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), name="maxpool_1")(x)

x = tf.keras.layers.Conv2D(
    256, (3, 3), activation="relu", padding="same", name="conv2d_2"
)(x)
x = tf.keras.layers.BatchNormalization(name="batchnorm_2")(x)
x = tf.keras.layers.Dropout(0.3, name="dropout_2")(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), name="maxpool_2")(x)

x = tf.keras.layers.Conv2D(
    128, (3, 3), activation="relu", padding="same", name="conv2d_3"
)(x)
x = tf.keras.layers.BatchNormalization(name="batchnorm_3")(x)
x = tf.keras.layers.Dropout(0.4, name="dropout_3")(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), name="maxpool_3")(x)

# Add GlobalAveragePooling2D before the Dense layers
x = tf.keras.layers.GlobalAveragePooling2D(name="globalaveragepool")(x)

# Add Dense layers
x = tf.keras.layers.Dense(512, activation="relu", name="dense_1")(x)
x = tf.keras.layers.Dropout(0.5, name="dropout_4")(x)
x = tf.keras.layers.Dense(128, activation="relu", name="dense_2")(x)

# Add output layer (classification head)
dummy_output = tf.keras.layers.Dense(
    2, activation="softmax", name="dummy_classification_head"
)(x)

# Wrap into a Keras model
wrapped_model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=dummy_output)

# https://github.com/keras-team/keras/blob/v3.3.3/keras/src/models/model.py#L217
# https://github.com/keras-team/keras/blob/master/keras/src/utils/summary_utils.py#L121
wrapped_model.summary(
    line_length=None,
    positions=None,
    print_fn=None,
    expand_nested=False,
    show_trainable=True,
    layer_range=None,
)

NameError: name 'transformer_model' is not defined

In [ ]:

# Visualize the wrapped model
img_nlp_with_tf_layers = visualkeras.layered_view(
    wrapped_model,
    legend=True,
    show_dimension=True,
    min_z=1,
    min_xy=1,
    max_z=4096,
    max_xy=4096,
    scale_z=1,
    scale_xy=1,
    font={"font_size": 99},
    text_callable="default",
    # to_file="result_images/nlp_with_tf_layers.png",
    save_fig=True,
    save_fig_filename="nlp_with_tf_layers.png",
    overwrite=False,
    add_timestamp=True,
    verbose=True,
)
img_nlp_with_tf_layers

# %%
#
# .. tags::
#
#    model-type: classification
#    model-workflow: model building
#    plot-type: visualkeras
#    domain: neural network
#    level: advanced
#    purpose: showcase